In [1]:
import os

In [2]:
%pwd

'c:\\Users\\manob\\Documents\\Kidney_Disease_Detection\\Kidney_Disease_Detection\\Pretrained Models\\VGG16'

In [3]:
os.chdir("../")
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\manob\\Documents\\Kidney_Disease_Detection\\Kidney_Disease_Detection'

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam


c:\Users\manob\Documents\Kidney_Disease_Detection\Kidney_Disease_Detection\.conda\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\manob\Documents\Kidney_Disease_Detection\Kidney_Disease_Detection\.conda\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\manob\Documents\Kidney_Disease_Detection\Kidney_Disease_Detection\.conda\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [6]:
# Config for Kidney Stone Detection Model
IMAGE_SIZE = (256, 256, 3)  # Standard input size for VGG16
LEARNING_RATE = 1e-4
SAVE_PATH = "Pretrained Models/VGG16/vgg16_kidney_stone_model.h5"
# Ensure directory exists
os.makedirs(os.path.dirname(SAVE_PATH), exist_ok=True)

In [7]:
# Load base model
base_model = VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=IMAGE_SIZE
)

# First, freeze all layers
base_model.trainable = False

# Then, unfreeze some of the deeper layers for fine-tuning
# Unfreeze the last 4 convolutional layers of VGG16
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Print the layers and their trainable status
print("Layers to be fine-tuned:")
for layer in base_model.layers:
    print(f"{layer.name}: {layer.trainable}")



Layers to be fine-tuned:
input_1: False
block1_conv1: False
block1_conv2: False
block1_pool: False
block2_conv1: False
block2_conv2: False
block2_pool: False
block3_conv1: False
block3_conv2: False
block3_conv3: False
block3_pool: False
block4_conv1: False
block4_conv2: False
block4_conv3: False
block4_pool: False
block5_conv1: True
block5_conv2: True
block5_conv3: True
block5_pool: True


In [8]:
# Add custom classification head for kidney stone detection
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling to reduce parameters
x = Dropout(0.5)(x)  # Increased dropout for better regularization
x = Dense(256, activation='relu')(x)  # Larger dense layer
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)  # Additional layer for more capacity
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification: stone vs non-stone

# Full model
model = Model(inputs=base_model.input, outputs=output)

In [9]:
# Compile model for binary classification
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',  # Standard loss for binary classification
    metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]  # Additional metrics for medical application
)

# Summary & Save
model.summary()
model.save(SAVE_PATH)
print(f" Kidney Stone Detection Model saved at: {SAVE_PATH}")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     